In [2]:
import json

f = open('res_info.json', 'r')
res_info = json.load(f)
f.close()

res_info

{'vender': 'aws',
 'clusterName': 'aws-eks-200214',
 'nodeGroupName': 'ng-200214',
 'workerRolearn': 'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5',
 'NodeInstanceRole': 'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5',
 'FileSystemId': 'fs-83a42529',
 'region': 'us-west-2',
 'db_address': 'rds-topse.ciph33ijhh38.us-west-2.rds.amazonaws.com',
 'vpcID': 'vpc-07b429d6780e6ec5f'}

# 削除

まずはALBIngressControllerIAMPolicyの削除

In [2]:
ALBPolicyArn = 'arn:aws:iam::565991835156:policy/ALBIngressControllerIAMPolicy'

In [3]:
workerRolearn = res_info['workerRolearn']

In [4]:
!aws iam detach-role-policy \
--policy-arn {ALBPolicyArn} \
--role-name {workerRolearn}

In [5]:
!aws iam delete-policy \
--policy-arn {ALBPolicyArn}

kubectlで作成したオブジェクトの削除

In [6]:
!kubectl delete -f yaml/wordpress-ingress.yaml
!kubectl delete -f yaml/wordpress-service.yaml
!kubectl delete -f yaml/wordpress-deploy.yaml
!kubectl delete -f yaml/alb-ingress-controller.yaml
!kubectl delete -f yaml/rbac-role.yaml
!kubectl delete -f yaml/deployment.yaml    

ingress.extensions "wordpress-ingress" deleted
service "wordpress" deleted
deployment.apps "wordpress" deleted
deployment.apps "alb-ingress-controller" deleted
clusterrole.rbac.authorization.k8s.io "alb-ingress-controller" deleted
clusterrolebinding.rbac.authorization.k8s.io "alb-ingress-controller" deleted
serviceaccount "alb-ingress-controller" deleted
serviceaccount "efs-provisioner" deleted
deployment.apps "efs-provisioner" deleted


In [9]:
!kubectl delete -f yaml/class.yaml
!kubectl delete -f yaml/configmap.yaml
!kubectl delete -f yaml/rbac.yaml
!kubectl delete -f yaml/deployment.yaml
!kubectl delete -f yaml/claim.yaml

Error from server (NotFound): error when deleting "yaml/class.yaml": storageclasses.storage.k8s.io "aws-efs" not found


RuntimeError: Unexpected exit code: 1

In [11]:
!aws efs describe-mount-targets \
--file-system-id {res_info['FileSystemId']}

{
    "MountTargets": [
        {
            "OwnerId": "565991835156",
            "MountTargetId": "fsmt-413e7de9",
            "FileSystemId": "fs-cc76c766",
            "SubnetId": "subnet-0880877de3ef6143a",
            "LifeCycleState": "available",
            "IpAddress": "10.0.97.63",
            "NetworkInterfaceId": "eni-0bb220dc84b6a7897"
        },
        {
            "OwnerId": "565991835156",
            "MountTargetId": "fsmt-423e7dea",
            "FileSystemId": "fs-cc76c766",
            "SubnetId": "subnet-0606ee1fa3fc64dff",
            "LifeCycleState": "available",
            "IpAddress": "10.0.89.105",
            "NetworkInterfaceId": "eni-0454599c821b9636d"
        }
    ]
}


In [13]:
!aws efs delete-mount-target \
--mount-target-id fsmt-413e7de9

In [14]:
!aws efs delete-mount-target \
--mount-target-id fsmt-423e7dea

In [15]:
!aws efs delete-file-system \
--file-system-id {res_info['FileSystemId']}

In [16]:
!aws iam detach-role-policy --role-name {res_info['NodeInstanceRole']}\
--policy-arn arn:aws:iam::aws:policy/AmazonElasticFileSystemFullAccess

クラスタの削除

In [ ]:
!eksctl delete cluster \
    --name {res_info['clusterName']} \
    --wait

[ℹ]  eksctl version 0.13.0
[ℹ]  using region us-west-2
[ℹ]  deleting EKS cluster "aws-eks-200130"
[ℹ]  account is not authorized to use Fargate. Ignoring error
[✔]  kubeconfig has been updated
[ℹ]  cleaning up LoadBalancer services
[ℹ]  2 sequential tasks: { delete nodegroup "ng-200130", delete cluster control plane "aws-eks-200130" }
[ℹ]  will delete stack "eksctl-aws-eks-200130-nodegroup-ng-200130"
[ℹ]  waiting for stack "eksctl-aws-eks-200130-nodegroup-ng-200130" to get deleted
[ℹ]  will delete stack "eksctl-aws-eks-200130-cluster"
[ℹ]  waiting for stack "eksctl-aws-eks-200130-cluster" to get deleted


RDS削除前の確認

In [3]:
!aws rds describe-db-instances \
        --db-instance-identifier "RDS-topse"

{
    "DBInstances": [
        {
            "DBInstanceIdentifier": "rds-topse",
            "DBInstanceClass": "db.t2.micro",
            "Engine": "mysql",
            "DBInstanceStatus": "available",
            "MasterUsername": "root",
            "Endpoint": {
                "Address": "rds-topse.ciph33ijhh38.us-west-2.rds.amazonaws.com",
                "Port": 3306,
                "HostedZoneId": "Z1PVIF0B656C1W"
            },
            "AllocatedStorage": 5,
            "InstanceCreateTime": "2020-02-14T08:44:07.189Z",
            "PreferredBackupWindow": "08:31-09:01",
            "BackupRetentionPeriod": 1,
            "DBSecurityGroups": [],
            "VpcSecurityGroups": [
                {
                    "VpcSecurityGroupId": "sg-08c8ea2cb2dfe07ca",
                    "Status": "active"
                }
            ],
            "DBParameterGroups": [
                {
                    "DBParameterGroupName": "default.mysql5.7",
                    "Par

RDS削除 

In [ ]:
!aws rds delete-db-instance --db-instance-identifier "RDS-topse" --skip-final-snapshot